# Backend

In [1]:
import numpy as np
from itertools import product
from collections import OrderedDict
from dataclasses import dataclass
import matplotlib.pyplot as plt
import math

In [2]:
significant_figures = 4
_float_tolerance = 5 * (10 ** -(significant_figures +1))

def _do_float_division_with_tolerance(divisor, dividend,):
    if _do_float_eq_with_tolerance(divisor, dividend):
        return int(1)
    
    quotient = np.true_divide(divisor, dividend)
    return quotient

def _do_float_subtraction_with_tolerance(minuend, subtrahend,):
    difference = np.subtract(minuend, subtrahend)

    if _do_float_le_with_tolerance(difference + _float_tolerance, int(difference)) and _do_float_ge_with_tolerance(difference - _float_tolerance, int(difference)):
        return int(difference)

    return difference

vectorized_float_division_with_tolerance = np.vectorize(_do_float_division_with_tolerance)
vectorized_float_subtraction_with_tolerance = np.vectorize(_do_float_subtraction_with_tolerance)

def _do_float_eq_with_tolerance(given, to_compare, sig_figs=significant_figures):
    tolerance = 5 * (10 ** -(significant_figures +1))
    return round(given - tolerance, sig_figs) <= to_compare and round(given + tolerance, sig_figs) >= to_compare

def _do_float_gt_with_tolerance(given, to_compare, sig_figs=significant_figures):
    tolerance = 5 * (10 ** -(significant_figures +1))
    return round(given - tolerance, sig_figs) > to_compare and round(given + tolerance, sig_figs) > to_compare

def _do_float_lt_with_tolerance(given, to_compare, sig_figs=significant_figures):
    tolerance = 5 * (10 ** -(significant_figures +1))
    return round(given - tolerance, sig_figs) < to_compare and round(given + tolerance, sig_figs) < to_compare

def _do_float_ge_with_tolerance(given, to_compare, sig_figs=significant_figures):
    return _do_float_gt_with_tolerance(given, to_compare, sig_figs) or _do_float_eq_with_tolerance(given, to_compare, sig_figs)

def _do_float_le_with_tolerance(given, to_compare, sig_figs=significant_figures):
    return _do_float_lt_with_tolerance(given, to_compare, sig_figs) or _do_float_eq_with_tolerance(given, to_compare, sig_figs)

vectorized_float_eq_with_tolerance = np.vectorize(_do_float_eq_with_tolerance)
vectorized_float_gt_with_tolerance = np.vectorize(_do_float_gt_with_tolerance)
vectorized_float_lt_with_tolerance = np.vectorize(_do_float_lt_with_tolerance)
vectorized_float_ge_with_tolerance = np.vectorize(_do_float_ge_with_tolerance)
vectorized_float_le_with_tolerance = np.vectorize(_do_float_le_with_tolerance)

In [3]:
M=1

def get_curvature_factor(r_coordinate):

    if r_coordinate == 0:
        return np.inf

    schwarzschild_radius = 2*M
    r_coordinate_scaled_by_M = np.multiply(r_coordinate, M)

    quotient = vectorized_float_division_with_tolerance(divisor=schwarzschild_radius, dividend=r_coordinate_scaled_by_M)
    difference = vectorized_float_subtraction_with_tolerance(minuend=np.ones(np.shape(r_coordinate)), subtrahend=quotient)

    if difference > 0:
        return np.sqrt(difference)
    else:
        return -np.sqrt(np.abs(difference))

def get_energy_per_unit_mass_at_shell(r_coordinate)->float:
    return _do_float_subtraction_with_tolerance(1, _do_float_division_with_tolerance(2*M, r_coordinate))

def V_effective(r_coordinate, angular_momentum_per_unit_mass)->float:
    return _do_float_subtraction_with_tolerance(1, _do_float_division_with_tolerance(2*M, r_coordinate))*_do_float_subtraction_with_tolerance(1, -_do_float_division_with_tolerance(angular_momentum_per_unit_mass, r_coordinate)**2)

def V_effective2(r_coordinate, delta_phi)->float:
    return _do_float_subtraction_with_tolerance(1, _do_float_division_with_tolerance(2*M, r_coordinate))*_do_float_subtraction_with_tolerance(1, - (r_coordinate**2)*delta_phi)

## The Stone

In [4]:
class Stone:
    def __init__(self, starting_coordinate, *, at_rest, energy_per_unit_mass=0, angular_momentum_per_unit_mass=0, dr_direction=-1):
        self.position = [starting_coordinate, ]
        self.dr_direction = dr_direction
        if at_rest:
            self.energy_per_unit_mass = get_energy_per_unit_mass_at_shell(self.position[0].polar.r)
            self.angular_momentum_per_unit_mass = angular_momentum_per_unit_mass
        else:
            self.energy_per_unit_mass = energy_per_unit_mass
            self.angular_momentum_per_unit_mass = angular_momentum_per_unit_mass
    
    def get_path_taken(self):
        t_elapsed = [t for t in np.arange(0, len(self.position))]
        path = self.position.copy()
        path.reverse()
        return path, t_elapsed

## Lattice

### Dimensions dataclass

In [5]:
@dataclass(frozen=True)
class Dimensions:
    left: float
    right: float
    top: float
    bottom: float
    resolution: float

@dataclass(frozen=True)
class PolarDimensions:
    inner_r: float
    outer_r: float
    r_resolution: float
    phi_resolution: float
    phi_sig_figs: int

### Coordinate and Coordinate System helper dataclasses

#### Cartesian

In [6]:
@dataclass(frozen=True)
class Cartesian:
    x: int
    y: int

    def __eq__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_eq_with_tolerance(self.x, other.x)
            y_match = vectorized_float_eq_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    def __ge__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_ge_with_tolerance(self.x, other.x)
            y_match = vectorized_float_ge_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    def __le__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_le_with_tolerance(self.x, other.x)
            y_match = vectorized_float_le_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    ## TODO: need to consider case where x == x, y > y
    def __gt__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_gt_with_tolerance(self.x, other.x)
            y_match = vectorized_float_gt_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_lt_with_tolerance(self.x, other.x)
            y_match = vectorized_float_lt_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

#### Polar

In [7]:
@dataclass(frozen=True)
class Polar:
    r: int
    phi: float

    def __eq__(self, other):
        if isinstance(other, type(self)):
            r_match = vectorized_float_eq_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_eq_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

    def __ge__(self, other):
        if isinstance(other, type(self)):
            r_match = vectorized_float_ge_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_ge_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

    def __le__(self, other):
        if isinstance(other, type(self)):
            r_match = vectorized_float_le_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_le_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

    def __gt__(self, other):
        if isinstance(other, type(self)):
            if vectorized_float_eq_with_tolerance(self.r, other.r):
                return vectorized_float_gt_with_tolerance(self.phi, other.phi)

            r_match = vectorized_float_gt_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_gt_with_tolerance(self.phi, other.phi)

            return r_match and phi_match
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, type(self)):
            if vectorized_float_eq_with_tolerance(self.r, other.r):
                return vectorized_float_lt_with_tolerance(self.phi, other.phi) 

            r_match = vectorized_float_lt_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_lt_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

#### Coordinates

In [8]:
class Coordinate:
    def __init__(self, q1: float, q2: float, is_cartesian=True,):
        if is_cartesian:
            self.cartesian = Cartesian(q1, q2)
            self.polar = Coordinate.convert_from_cartesian(self.cartesian)
        else:
            if _do_float_lt_with_tolerance(q2, 0):
                q2 = _do_float_subtraction_with_tolerance(q2, -2*np.pi)
            elif _do_float_ge_with_tolerance(q2, 2*np.pi):
                q2 = _do_float_subtraction_with_tolerance(q2, 2*np.pi)

            self.polar = Polar(q1, q2)
            self.cartesian = Coordinate.convert_from_polar(self.polar)
        self.curvature = get_curvature_factor(self.polar.r)
    
    def convert_from_polar(coordinates_to_convert: Polar):
        x_coordinate = np.multiply(coordinates_to_convert.r, np.cos(coordinates_to_convert.phi))
        y_coordinate = np.multiply(coordinates_to_convert.r, np.sin(coordinates_to_convert.phi))

        return Cartesian(x=x_coordinate, y=y_coordinate)

    def convert_from_cartesian(coordinates_to_convert: Cartesian):
        r_coordinate = np.sqrt(np.power(coordinates_to_convert.x , 2) + np.power(coordinates_to_convert.y , 2))

        if r_coordinate == 0:
            return Polar(r=r_coordinate, phi=0)
        
        phi_coordinate = np.arctan2(coordinates_to_convert.y, coordinates_to_convert.x)

        if _do_float_lt_with_tolerance(phi_coordinate, 0):
            phi_coordinate = _do_float_subtraction_with_tolerance(phi_coordinate, -2*np.pi)

        return Polar(r=r_coordinate, phi=phi_coordinate)
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.cartesian == other.cartesian and self.polar == other.polar
        else:
            return NotImplemented

    def __gt__(self, other):
        if isinstance(other, type(self)):
            return self.cartesian > other.cartesian and self.polar > other.polar
        else:
            return NotImplemented

    def __ge__(self, other):
        if isinstance(other, type(self)):
           return self.cartesian >= other.cartesian and self.polar >= other.polar
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, type(self)):
            return self.cartesian < other.cartesian and self.polar < other.polar
        else:
            return NotImplemented

    def __le__(self, other):
        if isinstance(other, type(self)):
           return self.cartesian <= other.cartesian and self.polar <= other.polar
        else:
            return NotImplemented


### The Latticework Class

#### Creation

In [18]:
class Latticework:

    def __init__(self, /, dimensions: Dimensions=None, polar_dimensions: PolarDimensions=None):
        self.mesh = None
        self.cartesian_dimensions = None
        self.polar_dimensions = None

        if not dimensions is  None:
            self.cartesian_dimensions = dimensions
            self._build_cartesian_grid()
        if not polar_dimensions is None:
            self.polar_dimensions = polar_dimensions
            self._build_polar_grid()
        # self._build_dictionary_of_shells()
    
    def _build_cartesian_grid(self):
        self.vertices = { }

        number_of_x_vertices = int((np.abs(self.cartesian_dimensions.left)+np.abs(self.cartesian_dimensions.right))/self.cartesian_dimensions.resolution) +1
        number_of_y_vertices = int(((np.abs(self.cartesian_dimensions.bottom)+np.abs(self.cartesian_dimensions.top)))/self.cartesian_dimensions.resolution) +1

        for x in np.linspace(start=self.cartesian_dimensions.left, stop=self.cartesian_dimensions.right, num=number_of_x_vertices, endpoint=True):
            for y in np.linspace(start=self.cartesian_dimensions.bottom, stop=self.cartesian_dimensions.top, num=number_of_y_vertices, endpoint=True):
                self.vertices.update({(x, y) : Coordinate(q1=x, q2=y)})
        self._build_meshgrid()

    def _build_polar_grid(self):

        number_of_r_vertices = int((np.abs(self.polar_dimensions.inner_r)+np.abs(self.polar_dimensions.outer_r))/self.polar_dimensions.r_resolution) +1
        number_of_phi_vertices = int((2*np.pi)/self.polar_dimensions.phi_resolution)

        R = np.linspace(start=self.polar_dimensions.inner_r, stop=self.polar_dimensions.outer_r, num=number_of_r_vertices, endpoint=True)
        Phi = np.linspace(start=0, stop=2*np.pi, num=number_of_phi_vertices, endpoint=False)

        R, Phi = np.meshgrid(R, Phi)
        self.mesh = R, Phi
        self.vertices = np.empty(R.shape, dtype=object).T


        it = np.nditer([R, Phi], flags=['multi_index', 'refs_ok'])

        for r, phi in it:
            r, phi = round(r[()], 1), round(phi[()], self.polar_dimensions.phi_sig_figs) #convert 0-Dim arrays to scalars
            iphi, ir = it.multi_index
            self.vertices[ir, iphi] = Coordinate(r, phi, is_cartesian=False)
        # self._build_meshgrid()

    def get_coordinates_of_constant_r(self, constant_r, ascending=True, complete_loop=True) -> list():
        shell_of_constant_r = self.shells["r"][constant_r].copy()
        if not ascending:
            shell_of_constant_r.sort(reverse= not ascending, key=lambda x: x.polar.phi)
        if complete_loop:
            shell_of_constant_r.append(shell_of_constant_r[0])
        return shell_of_constant_r
    
    def get_coordinates_of_constant_phi(self, constant_phi, ascending=True) -> list():
        shell_of_constant_phi = self.shells["phi"][constant_phi].copy()
        if not ascending:
            shell_of_constant_phi.sort(reverse= not ascending, key=lambda x: x.polar.r)

        return shell_of_constant_phi

    def _build_dictionary_of_shells(self):
        self.shells = { 
            "r" : { }, 
            "phi" : { }, 
            }

        sorted_vertices = list(self.vertices.values())
        sorted_vertices.sort(key=lambda x: x.polar.r)

        for coordinate in sorted_vertices:
            if coordinate.polar.r not in self.shells["r"].keys():
                self.shells["r"].update({coordinate.polar.r : [ coordinate, ]})
            else:
                self.shells["r"][coordinate.polar.r].append(coordinate)
        
        sorted_vertices.sort(key=lambda x: x.polar.phi)
        for coordinate in sorted_vertices:
            if coordinate.polar.phi not in self.shells["phi"].keys():
                self.shells["phi"].update({coordinate.polar.phi : [ coordinate, ]})
            else:
                self.shells["phi"][coordinate.polar.phi].append(coordinate)
            
        for r in self.shells["r"].keys():
            self.shells["r"][r].sort(key=lambda x: x.polar.phi)
        
        for phi in self.shells["phi"].keys():
            self.shells["phi"][phi].sort(key=lambda x: x.polar.r)
    
    def get_coordinate_at_edge(self):
        return self.shells["r"][self.dimensions.right][0]

    def _build_meshgrid(self):
        if not self.mesh is None:
            return

        if self.polar_dimensions is None:
            number_of_X_samples = int((np.abs(self.cartesian_dimensions.left)+np.abs(self.cartesian_dimensions.right))/self.cartesian_dimensions.resolution) +1
            number_of_Y_samples = int((np.abs(self.cartesian_dimensions.bottom)+np.abs(self.cartesian_dimensions.top))/self.cartesian_dimensions.resolution) +1

            X = np.linspace(start=self.cartesian_dimensions.left, stop=self.cartesian_dimensions.right, num=number_of_X_samples, endpoint=True)
            Y = np.linspace(start=self.cartesian_dimensions.bottom, stop=self.cartesian_dimensions.top, num=number_of_Y_samples, endpoint=True)
            Z = np.zeros((len(X), len(Y)))

            for i, (x,y) in enumerate(product(X,Y)):
                if _do_float_gt_with_tolerance(self.vertices[x,y].polar.r, 2*M + self.cartesian_dimensions.resolution, sig_figs=1):
                    Z[np.unravel_index(i, (len(X), len(Y)))] = np.true_divide(1, self.vertices[x,y].curvature)

            z_max_index = np.unravel_index(np.argmax(Z), Z.shape)
            Z = np.where(Z == 0, Z[z_max_index]+1, Z)

            X, Y = np.meshgrid(X, Y)

            self.mesh = (X, Y, Z)
            return
        
        if self.cartesian_dimensions is None:
            number_of_r_vertices = int((np.abs(self.polar_dimensions.inner_r)+np.abs(self.polar_dimensions.outer_r))/self.polar_dimensions.r_resolution) +1
            number_of_phi_vertices = int((2*np.pi)/self.polar_dimensions.phi_resolution)

            R = np.linspace(start=self.polar_dimensions.inner_r, stop=self.polar_dimensions.outer_r, num=number_of_r_vertices, endpoint=True)
            Phi = np.linspace(start=0, stop=2*np.pi, num=number_of_phi_vertices, endpoint=True)

            R, Phi = np.meshgrid(R, Phi)
            X, Y = R*np.cos(Phi), R*np.sin(Phi)

            Z = np.zeros(X.shape)
            offset = self.polar_dimensions.r_resolution/2 if self.polar_dimensions.r_resolution >= 1 else self.polar_dimensions.r_resolution

            it = np.nditer([X, Y], flags=['multi_index'])
            for x, y, in it:
                ix, iy = it.multi_index
                r = round(math.sqrt(math.pow(x,2) + math.pow(y,2))*M, int(math.fabs(math.log(self.polar_dimensions.r_resolution))))
                
                if r <= 2.0*M:
                    Z[ix, iy] = 1/get_curvature_factor(2*M+offset/2)
                else:
                    Z[ix, iy] = 1/get_curvature_factor(r)
                
            self.mesh = (X, Y, Z)
            return

        if not self.mesh is None:
            return self.mesh

        #Meshgrid Algorithm from: https://stackoverflow.com/a/32449311

        number_of_X_samples = int((np.abs(self.dimensions.left)+np.abs(self.dimensions.right))/self.dimensions.resolution) +1
        number_of_Y_samples = int((np.abs(self.dimensions.bottom)+np.abs(self.dimensions.top))/self.dimensions.resolution) +1

        X = np.linspace(start=self.dimensions.left, stop=self.dimensions.right, num=number_of_X_samples, endpoint=True)
        Y = np.linspace(start=self.dimensions.bottom, stop=self.dimensions.top, num=number_of_Y_samples, endpoint=True)
        Z = np.zeros((len(X), len(Y)))

        for i, (x,y) in enumerate(product(X,Y)):
            if _do_float_gt_with_tolerance(self.vertices[x,y].polar.r, 2*M + self.dimensions.resolution, sig_figs=1):
                Z[np.unravel_index(i, (len(X), len(Y)))] = np.true_divide(1, self.vertices[x,y].curvature)

        z_max_index = np.unravel_index(np.argmax(Z), Z.shape)
        Z = np.where(Z == 0, Z[z_max_index]+1, Z)

        X, Y = np.meshgrid(X, Y)

        self.mesh = (X, Y, Z)

    def _edge_cost(self, current: Polar, next: Polar, stone: Stone):
        v_eff = V_effective(next.r, stone.angular_momentum_per_unit_mass)
        if math.fabs(v_eff) > stone.energy_per_unit_mass ** 2:
            return np.nan

        # a = stone.angular_momentum_per_unit_mass
        # b = stone.angular_momentum_per_unit_mass/stone.energy_per_unit_mass

        # dphi = current.phi - next.phi
        # cost = ((next.r**4)/(b**2) - (1-2*M/next.r)*((next.r**4)/(a**2)+next.r**2))*(dphi**2)
        # sign = _do_float_division_with_tolerance(cost, math.fabs(cost))
        # return sign * math.sqrt(math.fabs(cost))
        
        energy_difference = stone.energy_per_unit_mass**2 - v_eff
        dtau = (next.r - current.r)/math.sqrt(math.fabs(energy_difference))
        if stone.dr_direction > 0:
            dtau *= -1
        sign = _do_float_division_with_tolerance(energy_difference, math.fabs(energy_difference))
        return sign * dtau

    def build_graph(self, stone: Stone)->None:
        self.graph = { }

        it = np.nditer(self.mesh, flags=['multi_index', 'refs_ok'])
        phi_max_index, r_max_index = it.shape

        for r, phi in it:

            #convert 0-Dim arrays to scalars
            r, phi = round(r[()], 1), round(phi[()], self.polar_dimensions.phi_sig_figs) 
            iphi, ir = it.multi_index

            vertex_to_get_neighbors_for = self.vertices[ir, iphi].polar

            # Skip everying inside the horizon
            if _do_float_le_with_tolerance(r, 2*M):
                continue

            self.graph.update({vertex_to_get_neighbors_for : [ ]})

            neighbors = [ ]
            for n_r, n_phi in product(range(ir-1, ir+2), range(iphi-1, iphi+2)):
                if n_r == r_max_index:
                    continue
                if n_phi == phi_max_index: #Wrap around
                    neighbors.append(self.vertices[n_r, n_phi-12].polar)
                else:
                    neighbors.append(self.vertices[n_r, n_phi].polar)
            for neighbor in neighbors:
                edge_cost = self._edge_cost(vertex_to_get_neighbors_for, neighbor, stone)
                if not np.isfinite(edge_cost):
                    continue
                angular_cost = 0
                delta_phi_remainder = math.remainder(neighbor.phi - phi, 2*np.pi)
                if delta_phi_remainder == 0:
                    angular_cost = -1*stone.angular_momentum_per_unit_mass
                if stone.angular_momentum_per_unit_mass > 0:
                    if delta_phi_remainder < 0:
                        angular_cost = -2*stone.angular_momentum_per_unit_mass
                else:
                    if delta_phi_remainder > 0:
                        angular_cost = -2*stone.angular_momentum_per_unit_mass

                self.graph[vertex_to_get_neighbors_for].append((neighbor, edge_cost-angular_cost))
            self.graph[vertex_to_get_neighbors_for].sort(key=lambda neighbor_cost_pair: neighbor_cost_pair[1])
    
    def find_path_for_stone(self, stone: Stone):

        energy_for_path = stone.energy_per_unit_mass **2
        total_cost = 0
        while len(stone.position) < 200:
            
            stone_position_polar = stone.position[0].polar
            possible_destinations = np.asarray(self.graph[Polar(round(stone_position_polar.r, 1), round(stone_position_polar.phi, self.polar_dimensions.phi_sig_figs)) ])
            # if stone.dr_direction > 0:
            #     possible_destinations[:,1] = np.multiply(possible_destinations[:,1], -1)
                # possible_destinations[:,1] = np.add(possible_destinations[:,1], 2)
            # for neighbor, cost in possible_destinations:
            #     print(f'{neighbor}, cost: {cost}')

            destinations_with_positive_flow = [possible_destinations[:,1] < 0]

            if possible_destinations[tuple(destinations_with_positive_flow)].shape == (0, 2):
                #change direction

                print(f'\nReached extrema radius ({stone_position_polar.r}, {stone_position_polar.phi}).\n')

                stone.dr_direction *= -1
                self.build_graph(stone)
                possible_destinations = np.asarray(self.graph[Polar(round(stone_position_polar.r, 1), round(stone_position_polar.phi, self.polar_dimensions.phi_sig_figs)) ])
                # possible_destinations[:,1] = np.multiply(possible_destinations[:,1], -1)
                # possible_destinations[:,1] = np.add(possible_destinations[:,1], 2)

                destinations_with_positive_flow = [possible_destinations[:,1] < 0]
                for neighbor, cost in possible_destinations:
                    print(f'{neighbor}, cost: {cost}')

            # print(possible_destinations[tuple(destinations_with_positive_flow)])

            step_taken, cost_of_step = possible_destinations[tuple(destinations_with_positive_flow)][0]
            total_cost -= cost_of_step
            stone.position.insert(0, Coordinate(step_taken.r, step_taken.phi, is_cartesian=False))

            print(f'Stepped to: {step_taken} with dtau cost of {-cost_of_step}.')
        print(f'\nElapsed proper time of path: {total_cost}')


# Dijkstra's Algorithm

In [19]:
stone = Stone(
        Coordinate(19.9, 0, is_cartesian=False), 
        at_rest=False,
        energy_per_unit_mass=math.sqrt(V_effective(r_coordinate=20, angular_momentum_per_unit_mass=4)),
        angular_momentum_per_unit_mass=4,
        dr_direction=-1
    )
lattice_dimensions = PolarDimensions(
        inner_r=0,
        outer_r=21,
        r_resolution=0.1,
        phi_resolution=0.5,
        phi_sig_figs=3
    )
lattice = Latticework(polar_dimensions=lattice_dimensions)
lattice.build_graph(stone=stone)

In [20]:
lattice.find_path_for_stone(stone=stone)

Stepped to: Polar(r=19.8, phi=0.524) with dtau cost of 5.5868572706252895.
Stepped to: Polar(r=19.7, phi=1.047) with dtau cost of 4.5604114442125905.
Stepped to: Polar(r=19.6, phi=1.571) with dtau cost of 3.9484278755521416.
Stepped to: Polar(r=19.5, phi=2.094) with dtau cost of 3.530745685760043.
Stepped to: Polar(r=19.4, phi=2.618) with dtau cost of 3.222411148509062.
Stepped to: Polar(r=19.3, phi=3.142) with dtau cost of 2.982777259859224.
Stepped to: Polar(r=19.2, phi=3.665) with dtau cost of 2.7896292001058143.
Stepped to: Polar(r=19.1, phi=4.189) with dtau cost of 2.629665171970965.
Stepped to: Polar(r=19.0, phi=4.712) with dtau cost of 2.4943695667561743.
Stepped to: Polar(r=18.9, phi=5.236) with dtau cost of 2.3780044271736074.
Stepped to: Polar(r=18.8, phi=5.76) with dtau cost of 2.276543532036298.
Stepped to: Polar(r=18.7, phi=0.0) with dtau cost of 2.187067268861788.
Stepped to: Polar(r=18.6, phi=0.524) with dtau cost of 2.1073999865275077.
Stepped to: Polar(r=18.5, phi=1.04

In [15]:
for r in [19.8, 19.9, 20.0, 20.1, 20.2]:
    # print(f'V={round(V_effective(r, 4), 4)},\t E-V={round(stone.energy_per_unit_mass**2 - V_effective(r, 4), 4)}')
    print(f'{1+(stone.angular_momentum_per_unit_mass/r)**2}')
print(stone.energy_per_unit_mass**2)

1.0408121620242832
1.0404030201257544
1.04
1.0396029801242543
1.0392118419762768
0.9359999999999999


In [160]:
for r in [8.1, 8.2, 8.3, 8.4, 8.5]:
    print(f'V={round(V_effective(r, 4), 4)},\t E-V={round(stone.energy_per_unit_mass**2 - V_effective(r, 4), 4)}')

V=0.9367,	 E-V=-0.0007
V=0.936,	 E-V=-0.0
V=0.9353,	 E-V=0.0007
V=0.9347,	 E-V=0.0013
V=0.9341,	 E-V=0.0019


In [21]:
stone = Stone(
        Coordinate(8.4, 4.189, is_cartesian=False), 
        at_rest=False,
        energy_per_unit_mass=math.sqrt(V_effective(r_coordinate=20, angular_momentum_per_unit_mass=4)),
        angular_momentum_per_unit_mass=4,
        dr_direction=1
    )
lattice_dimensions = PolarDimensions(
        inner_r=0,
        outer_r=21,
        r_resolution=0.1,
        phi_resolution=0.5,
        phi_sig_figs=3
    )
lattice = Latticework(polar_dimensions=lattice_dimensions)
lattice.build_graph(stone=stone)
lattice.find_path_for_stone(stone=stone)

Stepped to: Polar(r=8.5, phi=4.712) with dtau cost of 2.2660150958711167.
Stepped to: Polar(r=8.6, phi=5.236) with dtau cost of 1.986136658617633.
Stepped to: Polar(r=8.7, phi=5.76) with dtau cost of 1.7985001178008004.
Stepped to: Polar(r=8.8, phi=0.0) with dtau cost of 1.6624392677439404.
Stepped to: Polar(r=8.9, phi=0.524) with dtau cost of 1.5586101608299767.
Stepped to: Polar(r=9.0, phi=1.047) with dtau cost of 1.4764904075018614.
Stepped to: Polar(r=9.1, phi=1.571) with dtau cost of 1.4098051032591323.
Stepped to: Polar(r=9.2, phi=2.094) with dtau cost of 1.3545520276314167.
Stepped to: Polar(r=9.3, phi=2.618) with dtau cost of 1.3080437686059538.
Stepped to: Polar(r=9.4, phi=3.142) with dtau cost of 1.2684008733612655.
Stepped to: Polar(r=9.5, phi=3.665) with dtau cost of 1.2342648569205732.
Stepped to: Polar(r=9.6, phi=4.189) with dtau cost of 1.2046266201276028.
Stepped to: Polar(r=9.7, phi=4.712) with dtau cost of 1.1787191804110932.
Stepped to: Polar(r=9.8, phi=5.236) with d